In [2]:
import os 

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/ehsan/PycharmProjects/TextSummarizer/Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

    

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dirs


In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path: Path = CONFIG_FILE_PATH,
            params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_dirs([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        
        return data_transformation_config
    

In [8]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [16]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids']
        }
            
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))
        

In [18]:
try:
    config_manager = ConfigurationManager()
    data_transformation = DataTransformation(config_manager.get_data_transformation_config())
    data_transformation.convert()

except Exception as e:
    raise e

[2024-04-05 02:20:40,802: INFO: common:  Loaded config/config.yaml]
[2024-04-05 02:20:40,805: INFO: common:  Loaded params.yaml]
[2024-04-05 02:20:40,807: INFO: common:  create directory at artifacts]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 109255.19 examples/s]
